In [1]:
import torch

from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [2]:
from models.fuxi_vrwkv import FuxiV2VRWKV
from train_utils.dataloader_openstl import load_data

/home/asutkin/.conda/envs/vrwkv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using /home/asutkin/.cache/torch_extensions/py311_cu121 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/asutkin/.cache/torch_extensions/py311_cu121/wkv/build.ninja...
/home/asutkin/.conda/envs/vrwkv/lib/python3.11/site-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module wkv...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module wkv...


ninja: no work to do.


In [3]:
data_root = "/home/fa.buzaev/data_5/"
prediction_horizone = 12

In [4]:
dataloader_train, dataloader_vali, dataloader_test, mean, std = load_data(batch_size=50,
                                                                          val_batch_size=125,
                                                                          data_root=data_root,
                                                                          num_workers=6,
                                                                          # data_split='2_8125',
                                                                          # data_name='mv3',
                                                                          data_split='5_625',
                                                                          data_name='t2m',
                                                                          # data_name='mv6',
                                                                          # train_time=['1979', '2015'],
                                                                          train_time=['2010', '2015'],
                                                                          val_time=['2016', '2016'],
                                                                          test_time=['2017', '2018'],
                                                                          idx_in=[-11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1, 0],
                                                                          idx_out=[*range(1, prediction_horizone+1)],
                                                                          step=1,
                                                                          level=1,
                                                                          distributed=False, use_augment=False, 
                                                                          use_prefetcher=False, drop_last=False)

/home/asutkin/.conda/envs/vrwkv/lib/python3.11/site-packages/xarray/core/accessor_dt.py:441: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalendar().week instead.
  warnings.warn(
/home/asutkin/.conda/envs/vrwkv/lib/python3.11/site-packages/xarray/core/accessor_dt.py:441: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalendar().week instead.
  warnings.warn(
/home/asutkin/.conda/envs/vrwkv/lib/python3.11/site-packages/xarray/core/accessor_dt.py:441: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalendar().week instead.
  warnings.warn(
/home/asutkin/.conda/envs/vrwkv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoade

In [6]:
torch.cuda.empty_cache()

In [ ]:
from tqdm import tqdm

criterion = torch.nn.MSELoss()
model = FuxiV2VRWKV(img_size=(1, 32, 64), patch_size=(1, 4, 4), in_chans=12, out_chans=1, embed_dim=64, num_groups=16, num_heads=8, window_size=7).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.5)
for i in range(10):
    model.train()
    running_loss = 0
    for x_train, y_train in tqdm(dataloader_train):
        x_train, y_train = x_train.to(device), y_train.to(device)
        optimizer.zero_grad()
        big_loss = 0
        for i in range(1, prediction_horizone):
            time_tensor = i * torch.ones(x_train.shape[0], device=x_train.device).unsqueeze(-1)
            prediction = model(x_train, time_tensor)
            loss = criterion(prediction, y_train[:,i,:,:,:])
            torch.cat((x_train[:,1:,:,:,:], prediction.unsqueeze(1)), dim=1)
            big_loss += loss
        big_loss.backward()
        optimizer.step()
        running_loss += big_loss.item() / prediction_horizone
    print(running_loss / len(dataloader_train))

100%|██████████| 1051/1051 [41:25<00:00,  2.37s/it]


0.021154796851747495


 13%|█▎        | 132/1051 [04:55<34:06,  2.23s/it]